In [1]:
!pip install sentence_transformers pypdf
!pip install langchain langchain-openai
!pip install -qU langchain_community faiss-cpu
!pip install transformers
!pip install langchain_community
!pip install sentence-transformers

In [52]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.documents import Document
import faiss
import os
from uuid import uuid4

In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool

In [54]:
import os

os.environ["OPENAI_API_KEY"] = "" # YOUR OPENAI_API_KEY
os.environ["LANGCHAIN_API_KEY"] = "" # YOUR LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "aviation"

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

 = ""
model_name = 'text-embedding-ada-002'
embeddings = OpenAIEmbeddings(model=model_name, openai_api_key=openai_api_key)

# 初始化Pinecone客户端
pinecone_api_key = "cd107fc4-b21c-48a1-afde-b5cf207d52e2"
pc = Pinecone(
    api_key=pinecone_api_key,
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
index_name = 'aviation-knowledge-base-0815'

# 连接到现有的Pinecone索引
index = pc.Index(index_name)

# 创建Pinecone向量存储实例
vectorstore = PineconeVectorStore(index=index, embedding=embeddings)

In [55]:
# 初始化嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5", 
                                   encode_kwargs={"normalize_embeddings": True})

# 生成一个测试文本的嵌入，以确定嵌入向量的维度
test_vector = embeddings.embed_query("hello world")
# 使用嵌入向量的长度初始化 FAISS 索引
index = faiss.IndexFlatL2(len(test_vector))

# 初始化 FAISS 向量存储
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [56]:
#show the list
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 使用合适的嵌入模型，例如 OpenAIEmbeddings 或其他你使用的嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5", 
                                   encode_kwargs={"normalize_embeddings": True})

# 加载 vector_store，并允许危险的反序列化
vector_store = FAISS.load_local("vectorstore.db", embeddings, allow_dangerous_deserialization=True)

# 获取向量存储中所有文档的 metadata
def get_all_metadata(vector_store):
    all_metadata = []
    
    # 检索向量存储中的所有文档
    documents = vector_store.similarity_search(query="", k=vector_store.index.ntotal)
    
    # 提取每个文档的 metadata
    for doc in documents:
        all_metadata.append(doc.metadata)
    
    return all_metadata

# 调用函数获取并查看所有 metadata
metadata_list = get_all_metadata(vector_store)
for metadata in metadata_list:
    print(metadata)

RuntimeError: Error in __cdecl faiss::FileIOReader::FileIOReader(const char *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\impl\io.cpp:68: Error: 'f' failed: could not open vectorstore.db\index.faiss for reading: No such file or directory

In [46]:
# 定义上传和处理文档的函数
def upload_document(file_path, metadata):
    # 加载 PDF
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    
    # 为每个文档添加指定的元数据
    for i, doc in enumerate(documents):
        documents[i] = Document(
            page_content=doc.page_content,
            metadata=metadata
        )

    # 将文档拆分成小块
    text = RecursiveCharacterTextSplitter().split_documents(documents)
    
    # 生成文档的 UUID 并添加到向量存储中
    uuids = [str(uuid4()) for _ in range(len(text))]
    vector_store.add_documents(documents=text, ids=uuids)

    # 保存更新后的向量存储
    vector_store.save_local("vectorstore.db")

# 使用函数上传文档并添加元数据
upload_document("Files\Knowledge\Mission File_Steep Turn.pdf", {"source": "Files\Knowledge\Mission File_Steep Turn.pdf", "category": "Steep Turn","importance": "1"})

<>:25: SyntaxWarning: invalid escape sequence '\K'
<>:25: SyntaxWarning: invalid escape sequence '\K'
<>:25: SyntaxWarning: invalid escape sequence '\K'
<>:25: SyntaxWarning: invalid escape sequence '\K'
C:\Users\888\AppData\Local\Temp\ipykernel_6312\3983081657.py:25: SyntaxWarning: invalid escape sequence '\K'
  upload_document("Files\Knowledge\Mission File_Steep Turn.pdf", {"source": "Files\Knowledge\Mission File_Steep Turn.pdf", "category": "Steep Turn","importance": "1"})
C:\Users\888\AppData\Local\Temp\ipykernel_6312\3983081657.py:25: SyntaxWarning: invalid escape sequence '\K'
  upload_document("Files\Knowledge\Mission File_Steep Turn.pdf", {"source": "Files\Knowledge\Mission File_Steep Turn.pdf", "category": "Steep Turn","importance": "1"})


In [59]:
!pip install unstructured
!pip install markdown

import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.vectorstores import FAISS
from langchain.schema import Document
from uuid import uuid4

# 定义上传和处理 Markdown 文件的函数
def upload_markdown(file_path, metadata):
    # 加载 Markdown 文件
    loader = UnstructuredMarkdownLoader(file_path, mode="elements")
    documents = loader.load()

    # 为每个文档添加指定的元数据
    for i, doc in enumerate(documents):
        documents[i] = Document(
            page_content=doc.page_content,
            metadata=metadata
        )
    
    # 生成文档的 UUID 并添加到向量存储中
    uuids = [str(uuid4()) for _ in range(len(documents))]
    vector_store.add_documents(documents=documents, ids=uuids)

    # 保存更新后的向量存储
    vector_store.save_local("vectorstore.db")

# 使用函数上传 Markdown 文件并添加元数据
upload_markdown("RAG_steep turns\Mission File_Steep Turn.md", {"source": "Mission File_Steep Turn.md", "category": "Steep turn"})

<>:34: SyntaxWarning: invalid escape sequence '\M'
<>:34: SyntaxWarning: invalid escape sequence '\M'
C:\Users\888\AppData\Local\Temp\ipykernel_17680\478075916.py:34: SyntaxWarning: invalid escape sequence '\M'
  upload_markdown("RAG_steep turns\Mission File_Steep Turn.md", {"source": "Mission File_Steep Turn.md", "category": "Steep turn"})


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\888\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\888\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [ ]:
!pip install unstructured
!pip install markdown

import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.vectorstores import FAISS
from langchain.schema import Document
from uuid import uuid4

# 定义上传和处理 Markdown 文件的函数
def upload_markdown(file_path, metadata):
    # 加载 Markdown 文件
    loader = UnstructuredMarkdownLoader(file_path, mode="elements")
    documents = loader.load()

    # 为每个文档添加指定的元数据
    for i, doc in enumerate(documents):
        documents[i] = Document(
            page_content=doc.page_content,
            metadata=metadata
        )
    
    # 生成文档的 UUID 并添加到向量存储中
    uuids = [str(uuid4()) for _ in range(len(documents))]
    vector_store.add_documents(documents=documents, ids=uuids)

    # 保存更新后的向量存储
    vector_store.save_local("vectorstore.db")

# 使用函数上传 Markdown 文件并添加元数据
upload_markdown("Files\Knowledge\Flight Maneuvers.md", {"source": "Knowledge/Flight Maneuvers.md", "category": "Knowledge", "importance": "2"})

In [9]:
from langchain.vectorstores import FAISS
# 使用合适的嵌入模型，例如 OpenAIEmbeddings 或其他你使用的嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5", 
                                   encode_kwargs={"normalize_embeddings": True})

# 加载 vector_store，并允许危险的反序列化
vector_store = FAISS.load_local("vectorstore.db", embeddings, allow_dangerous_deserialization=True)

# Create retriever
retriever = vector_store.as_retriever()

In [10]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import OpenAI
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool

In [37]:
flight_data_yaml = """
timestamp: "2024-08-04 15:20:56"
data:
  ACCELERATION_BODY_X: 0.0
  ACCELERATION_BODY_Y: 0.0
  ACCELERATION_BODY_Z: 0.0
  AIRSPEED_INDICATED: 0.045333489775657654
  AIRSPEED_TRUE: 0.04534189775586128
  GROUND_ALTITUDE: 2.2645655255764723
  GROUND_VELOCITY: 0.0
  INCIDENCE_ALPHA: -0.05926802009344101
  INCIDENCE_BETA: 0.0
  PLANE_ALTITUDE: 11.125602913606588
  PLANE_BANK_DEGREES: -0.0016650896301865436
  PLANE_HEADING_DEGREES_MAGNETIC: 0.06431762168901287
  PLANE_HEADING_DEGREES_TRUE: 6.23780079622633
  PLANE_PITCH_DEGREES: 0.00394632125321473
  VERTICAL_SPEED: 0.0
"""

# 解析YAML格式的数据
flight_data = yaml.safe_load(flight_data_yaml)

# 将解析后的数据转换为字符串
flight_data_str = str(flight_data)

print(flight_data_str)

{'timestamp': '2024-08-04 15:20:56', 'data': {'ACCELERATION_BODY_X': 0.0, 'ACCELERATION_BODY_Y': 0.0, 'ACCELERATION_BODY_Z': 0.0, 'AIRSPEED_INDICATED': 0.045333489775657654, 'AIRSPEED_TRUE': 0.04534189775586128, 'GROUND_ALTITUDE': 2.2645655255764723, 'GROUND_VELOCITY': 0.0, 'INCIDENCE_ALPHA': -0.05926802009344101, 'INCIDENCE_BETA': 0.0, 'PLANE_ALTITUDE': 11.125602913606588, 'PLANE_BANK_DEGREES': -0.0016650896301865436, 'PLANE_HEADING_DEGREES_MAGNETIC': 0.06431762168901287, 'PLANE_HEADING_DEGREES_TRUE': 6.23780079622633, 'PLANE_PITCH_DEGREES': 0.00394632125321473, 'VERTICAL_SPEED': 0.0}}


In [13]:
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain_community.llms import Ollama
from langchain.memory import ConversationBufferWindowMemory

In [31]:
# 解析YAML格式的数据
flight_data = """{'timestamp': '2024-08-24T15:30:57.342Z', 'plane_altitude': '2400.9382626', 'ground_altitude': '217.0375361', 'plane_pitch_degrees': '-2.1871284', 'plane_bank_degrees': '-41.9298986', 'plane_heading_degrees_true': '257.0383409', 'plane_heading_degrees_magnetic': '266.2906136', 'airspeed_indicated': '116.6328278', 'airspeed_true': '120.8094788', 'ground_velocity': '119.0164008', 'vertical_speed': '200.3418303', 'incidence_beta': '0.1352000', 'acceleration_body_x': '21.5425801', 'acceleration_body_y': '16.1129594', 'acceleration_body_z': '0.0299524', 'incidence_alpha': '1.6090642'}"""

# 初始化检索器
retriever = vector_store.as_retriever(k=3)

# 从hub加载prompt
prompt = hub.pull("aviation")

@tool
def judge_flight_status(flight_data: str) -> str:
    """You will be given flight data. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. Return the current flight status as a string."""
    flight_status = f"analyze_flight_data: {flight_data}"
    return flight_status

@tool
def retrieve_guidance(flight_status: str) -> str:
    """Based on the determined flight status, query the vector store to retrieve appropriate operational guidance."""
    guidance_query = f"Retrieve guidance for {flight_status}."
    message = retriever.invoke(guidance_query)
    # Extract page content from each document in the message
    guidance = [doc.page_content for doc in message]
    # Convert the guidance list into a single string, joining with a suitable delimiter (e.g., newline, comma)
    guidance_str = "\n".join(guidance)  # Here, newline is used as a delimiter
    return guidance_str

@tool
def generate_answer(answer: str) -> str:
    """
    Based on the retrieved operational guidance, recommend specific joystick movements to optimize flight performance according to the current phase and maneuver.
    Output the required joystick movements as follows:
    For pitch adjustments: "pitch = forward/back/no"
    For roll adjustments: "roll = left/right/no"
    """
    return f"Here is the answer to the user question: {answer}"

# Register tools
tools = [judge_flight_status, retrieve_guidance, generate_answer]

# 初始化 LLM
llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)
# llm = OpenAI(base_url="http://localhost:211/v1", api_key="lm-studio")
# llm = Ollama(model="llama3.1")

# 创建 agent
agent = create_react_agent(llm, tools, prompt)

# 创建 agent 执行器
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# 示例 flight data
# 你可以将 flight_data 直接传入
agent_executor.invoke({
    "input": flight_data
})



> Entering new AgentExecutor chain...
To answer the question, I need to determine the current flight status based on the provided flight data. The key parameters to analyze are the angle of bank and heading.

Action: judge_flight_status
Action Input: {'plane_bank_degrees': '-41.9298986', 'plane_heading_degrees_true': '257.0383409'}analyze_flight_data: {'plane_bank_degrees': '-41.9298986', 'plane_heading_degrees_true': '257.0383409'}To determine the current flight status, I need to analyze the angle of bank and heading. The plane is currently in a left bank of approximately 42°, which is close to the required 45° for a steep turn. The heading indicates the direction the plane is facing.

Action: retrieve_guidance
Action Input: "left steep turn"5. Steep Turn Techniques
6. Rolling Out of a Turn
Establishing a Turn in Flight
4. Monitoring and Correcting During the TurnTo generate the final answer, I need to provide specific joystick movements and adjustments based on the retrieved operat

{'input': "{'timestamp': '2024-08-24T15:30:57.342Z', 'plane_altitude': '2400.9382626', 'ground_altitude': '217.0375361', 'plane_pitch_degrees': '-2.1871284', 'plane_bank_degrees': '-41.9298986', 'plane_heading_degrees_true': '257.0383409', 'plane_heading_degrees_magnetic': '266.2906136', 'airspeed_indicated': '116.6328278', 'airspeed_true': '120.8094788', 'ground_velocity': '119.0164008', 'vertical_speed': '200.3418303', 'incidence_beta': '0.1352000', 'acceleration_body_x': '21.5425801', 'acceleration_body_y': '16.1129594', 'acceleration_body_z': '0.0299524', 'incidence_alpha': '1.6090642'}",
 'output': 'For maintaining the steep turn, use the following joystick movements: pitch = no, roll = left. This will help maintain the current left steep turn while keeping the bank angle close to 45°.'}

In [ ]:
# 解析YAML格式的数据
flight_data = """{'timestamp': '2024-08-24T15:31:42.625Z', 'plane_altitude': '2339.7345046', 'ground_altitude': '127.4123033', 'plane_pitch_degrees': '-2.3305856', 'plane_bank_degrees': '45.4910839', 'plane_heading_degrees_true': '25.2983926', 'plane_heading_degrees_magnetic': '34.2305463', 'airspeed_indicated': '122.4056396', 'airspeed_true': '126.5905533', 'ground_velocity': '127.5915478', 'vertical_speed': '16.6264361', 'incidence_beta': '0.2434822', 'acceleration_body_x': '-23.0330251', 'acceleration_body_y': '32.0754133', 'acceleration_body_z': '0.5025474', 'incidence_alpha': '2.9207101'}"""

# 初始化检索器
retriever = vector_store.as_retriever(k=3)

# Initialize memory
memory = ConversationBufferWindowMemory(k=500, memory_key="chat_history", return_messages=True)

# 从hub加载prompt
prompt = hub.pull("aviation")

@tool
def judge_flight_status(flight_data: str) -> str:
    """You will be given flight data. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. Return the current flight status as a string."""
    flight_status = f"analyze_flight_data: {flight_data}"
    return flight_status

@tool
def retrieve_guidance(flight_status: str) -> str:
    """Based on the determined flight status, query the vector store to retrieve appropriate operational guidance."""
    guidance_query = f"Retrieve guidance for {flight_status}."
    message = retriever.invoke(guidance_query)
    # Extract page content from each document in the message
    guidance = [doc.page_content for doc in message]
    # Convert the guidance list into a single string, joining with a suitable delimiter (e.g., newline, comma)
    guidance_str = "\n".join(guidance)  # Here, newline is used as a delimiter
    return guidance_str

@tool
def generate_answer(answer: str) -> str:
    """
    Based on the retrieved operational guidance, recommend specific joystick movements to optimize flight performance according to the current phase and maneuver.
    Output the required joystick movements as follows:
    For flight status:"flight_status={flight_status}"
    For pitch adjustments: "pitch = forward/back/no"
    For roll adjustments: "roll = left/right/no"
    """
    return f"Here is the answer to the user question: {answer}"

# Register tools
tools = [judge_flight_status, retrieve_guidance, generate_answer]

# 初始化 LLM
llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)
# llm = OpenAI(base_url="http://localhost:211/v1", api_key="lm-studio")
# llm = Ollama(model="llama3.1")

# 创建 agent
agent = create_react_agent(llm, tools, prompt)

# 创建 agent 执行器
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, handle_parsing_errors=True)

# 示例 flight data
# 你可以将 flight_data 直接传入
agent_executor.invoke({
    "input": flight_data
})

In [68]:
#状态
# 解析YAML格式的数据
flight_data = """{'timestamp': '2024-08-24T15:31:42.625Z', 'plane_altitude': '2339.7345046', 'ground_altitude': '127.4123033', 'plane_pitch_degrees': '-2.3305856', 'plane_bank_degrees': '45.4910839', 'plane_heading_degrees_true': '25.2983926', 'plane_heading_degrees_magnetic': '34.2305463', 'airspeed_indicated': '122.4056396', 'airspeed_true': '126.5905533', 'ground_velocity': '127.5915478', 'vertical_speed': '16.6264361', 'incidence_beta': '0.2434822', 'acceleration_body_x': '-23.0330251', 'acceleration_body_y': '32.0754133', 'acceleration_body_z': '0.5025474', 'incidence_alpha': '2.9207101'}"""

# 初始化检索器
retriever = vector_store.as_retriever(k=3)

# Initialize memory
memory = ConversationBufferWindowMemory(k=500, memory_key="chat_history", return_messages=True)

# 从hub加载prompt
prompt = hub.pull("aviation")

@tool
def judge_flight_status(flight_data: str) -> str:
    """You will be given flight data. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. Return the current flight status as a string."""
    flight_status = f"analyze_flight_data: {flight_data}"
    return flight_status

@tool
def retrieve_guidance(flight_status: str) -> str:
    """Based on the determined flight status, query the vector store to retrieve appropriate operational guidance."""
    guidance_query = f"Retrieve guidance for {flight_status}."
    message = retriever.invoke(guidance_query)
    # Extract page content from each document in the message
    guidance = [doc.page_content for doc in message]
    # Convert the guidance list into a single string, joining with a suitable delimiter (e.g., newline, comma)
    guidance_str = "\n".join(guidance)  # Here, newline is used as a delimiter
    return guidance_str

@tool
def generate_answer(answer: str) -> str:
    """
    Based on the retrieved operational guidance, recommend specific joystick movements to optimize flight performance according to the current phase and maneuver.
    Output the required joystick movements as follows:
    For flight status:"flight_status={flight_status}"
    For pitch adjustments: "pitch = forward/back/no"
    For roll adjustments: "roll = left/right/no"
    """
    return f"Here is the answer to the user question: {answer}"

# Register tools
tools = [judge_flight_status, retrieve_guidance, generate_answer]

# 初始化 LLM
llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)
# llm = OpenAI(base_url="http://localhost:211/v1", api_key="lm-studio")
# llm = Ollama(model="llama3.1")

# 创建 agent
agent = create_react_agent(llm, tools, prompt)

# 创建 agent 执行器
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, handle_parsing_errors=True)

# 示例 flight data
# 你可以将 flight_data 直接传入
agent_executor.invoke({
    "input": flight_data
})

import re

def extract_flight_controls(answer: str) -> str:
    # 正则表达式模式，用于匹配 flight_status, pitch 和 roll
    flight_status_pattern = r'flight_status=["\']([\w_]+)["\']'
    pitch_pattern = r"pitch\s*=\s*(\w+)"
    roll_pattern = r"roll\s*=\s*(\w+)"

    # 使用正则表达式进行匹配
    flight_status_match = re.search(flight_status_pattern, answer)
    pitch_match = re.search(pitch_pattern, answer)
    roll_match = re.search(roll_pattern, answer)

    # 提取出匹配到的值
    flight_status = flight_status_match.group(1) if flight_status_match else "Unknown"
    pitch = pitch_match.group(1) if pitch_match else "no"
    roll = roll_match.group(1) if roll_match else "no"

    # 构建返回的字符串
    result = f"flight_status={flight_status}, pitch={pitch}, roll={roll}"
    return result


# 提取并打印结果
extracted_controls = extract_flight_controls(answer)
print(extracted_controls)



> Entering new AgentExecutor chain...
To answer the question, I need to determine the current phase of the steep turn using the provided flight data. I will first analyze the flight data to determine the current flight status.

Action: judge_flight_status
Action Input: {'plane_bank_degrees': '45.4910839', 'plane_heading_degrees_true': '25.2983926'}analyze_flight_data: {'plane_bank_degrees': '45.4910839', 'plane_heading_degrees_true': '25.2983926'}To determine the current flight status, I need to analyze the angle of bank and heading from the flight data. The plane is currently in a steep turn with a bank angle of approximately 45 degrees and a heading of about 25 degrees true. This indicates that the plane is in the process of executing a 45° steep turn to the left.

Action: retrieve_guidance
Action Input: "flight_status=steep_turn_left"Establishing a Turn in Flight
Flight Maneuvers
Flight Controls
Straight Flight: Maintaining a constant direction or heading involves visually checkin

In [10]:
import serial
import time
import os
import json
import re
from typing import Literal
import langchain
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory

# 设置串口端口和波特率（根据你的情况修改） 
ser = serial.Serial('COM1', 9600)  # 替换为你实际的串口端口
#time.sleep(2)  # 等待串口连接稳定

# 文件路径
file_path = "simulator_data_1s1.txt"
#output_json_path = r'C:\Users\888\Desktop\Aviation\responses.json'

# 初始化文件大小
last_file_size = 0

# Initialize the OpenAI model
llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)

langchain.verbose = True

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# First prompt: Determine flight status
first_prompt = ChatPromptTemplate.from_template(
    """You will be given {flight_data}. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. 
    Return the current flight status as a string."""
)

chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="flight_status",
)

rag_prompt = ChatPromptTemplate.from_template(
    """
    Based on the {flight_status}, retrieve the operational and attention guidance.
    """
)


# Second chain: Retrieve guidance based on flight status
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    chain_type_kwargs={"prompt": rag_prompt},
    input_key="flight_status",
    output_key="retrieved_guidance"
)


# Third prompt: Recommend joystick movements based on retrieved guidance
third_prompt = ChatPromptTemplate.from_template(
    """
    Based on the {retrieved_guidance}, recommend specific joystick movements and the instrument to focus to optimize flight performance according to the current phase and maneuver.
    Output the required joystick movements as follows:
    For pitch adjustments: "pitch = forward/back/no"
    For roll adjustments: "roll = left/right/no"
    For instrument to focus: "instrument = Altimeter/Airspeed Indicator/Altimeter/Heading Indicator"
    """
)

chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="answer",
)

# Set up the overall SequentialChain
overall_chain = SequentialChain(
    chains=[chain_one, qa, chain_three],
    input_variables=["flight_data"],
    output_variables=["flight_status", "retrieved_guidance", "answer"],
    verbose=True
)


# 函数：发送消息并获取响应
def send_message_to_llm(flight_data: str):
    try:
        # 调用 agent_executor 来获取响应
        response =  overall_chain({"flight_data": flight_data})

        # 从响应中提取控制指令
        extracted_controls = extract_flight_controls(response)

        # 打印提取结果
        print("Response from LLM:")
        print(extracted_controls)

        # 发送消息到Arduino
        messagesending_str = json.dumps(extracted_controls, ensure_ascii=False)
        ser.write(messagesending_str.encode('utf-8'))
        print(f"Sent: {messagesending_str}")

    except Exception as e:
        print(f"Failed to get a response from the LLM. Error: {e}")

# 持续监控文件更新
while True:
    try:
        # 获取当前文件大小
        current_file_size = os.path.getsize(file_path)
        
        # 检查文件大小是否增加，表示有新内容
        if current_file_size > last_file_size:
            # 打开文件并读取新内容
            with open(file_path, 'r') as file:
                # 将光标移到上次读取的位置
                file.seek(last_file_size)
                # 读取新内容
                new_content = file.read()
                
                # 将内容拆分为行（假设每个数据是一个新行）
                lines = new_content.splitlines()
                
                # 处理每一行
                for line in lines:
                    # 单独发送每一行
                    if line.strip():  # 检查行是否不为空
                        print(f"Sending new content to LLM: {line.strip()}")
                        send_message_to_llm(line.strip())
                
                # 更新上次读取的文件大小
                last_file_size = current_file_size
    
    except Exception as e:
        print(f"An error occurred: {e}")
        break

c:\Users\888\.conda\envs\aviation\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


ValidationError: 1 validation error for StuffDocumentsChain
__root__
  document_variable_name context was not found in llm_chain input_variables: ['flight_status'] (type=value_error)

In [ ]:
import serial
import time
import os
import json
import re
from typing import Literal
from pydantic import BaseModel, Field, conint
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 设置串口端口和波特率（根据你的情况修改） 
ser = serial.Serial('COM12', 9600)  # 替换为你实际的串口端口
#time.sleep(2)  # 等待串口连接稳定

# 文件路径
file_path = "simulator_data_1s1.txt"
#output_json_path = r'C:\Users\888\Desktop\Aviation\responses.json'

# 初始化文件大小
last_file_size = 0

# 初始化检索器
retriever = vector_store.as_retriever(k=3)

# Initialize memory
memory = ConversationBufferWindowMemory(k=500, memory_key="chat_history", return_messages=True)

# 从hub加载prompt
prompt = hub.pull("aviation")

# 工具函数定义
@tool
def judge_flight_status(flight_data: str) -> str:
    """You will be given flight data. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. Return the current flight status as a string."""
    flight_status = f"analyze_flight_data: {flight_data}"
    return flight_status

@tool
def retrieve_guidance(flight_status: str) -> str:
    """Based on the determined flight status, query the vector store to retrieve appropriate operational guidance."""
    guidance_query = f"Retrieve guidance for {flight_status}."
    message = retriever.invoke(guidance_query)
    # Extract page content from each document in the message
    guidance = [doc.page_content for doc in message]
    # Convert the guidance list into a single string, joining with a suitable delimiter (e.g., newline, comma)
    guidance_str = "\n".join(guidance)  # Here, newline is used as a delimiter
    return guidance_str

@tool
def generate_answer(answer: str) -> str:
    """
    Based on the retrieved operational guidance, recommend specific joystick movements to optimize flight performance according to the current phase and maneuver.
    Output the required joystick movements as follows:
    For flight status:"flight_status={flight_status}"
    For pitch adjustments: "pitch = forward/back/no"
    For roll adjustments: "roll = left/right/no"
    """
    return f"Here is the answer to the user question: {answer}"

# Register tools
tools = [judge_flight_status, retrieve_guidance, generate_answer]

# 初始化 LLM
llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)

# 创建 agent
agent = create_react_agent(llm, tools, prompt)

# 创建 agent 执行器
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, handle_parsing_errors=True)

# 提取flight controls的函数
def extract_flight_controls(answer: str) -> str:
    # 正则表达式模式，用于匹配 flight_status, pitch 和 roll
    flight_status_pattern = r'flight_status=["\']([\w_]+)["\']'
    pitch_pattern = r"pitch\s*=\s*(\w+)"
    roll_pattern = r"roll\s*=\s*(\w+)"

    # 使用正则表达式进行匹配
    flight_status_match = re.search(flight_status_pattern, answer)
    pitch_match = re.search(pitch_pattern, answer)
    roll_match = re.search(roll_pattern, answer)

    # 提取出匹配到的值
    flight_status = flight_status_match.group(1) if flight_status_match else "Unknown"
    pitch = pitch_match.group(1) if pitch_match else "no"
    roll = roll_match.group(1) if roll_match else "no"

    # 构建返回的字符串
    result = f"flight_status={flight_status}, pitch={pitch}, roll={roll}"
    return result

# 函数：发送消息并获取响应
def send_message_to_llm(flight_data: str):
    try:
        # 调用 agent_executor 来获取响应
        response = agent_executor.invoke({
            "input": flight_data
        })

        # 从响应中提取控制指令
        extracted_controls = extract_flight_controls(response)

        # 打印提取结果
        print("Response from LLM:")
        print(extracted_controls)

        # 发送消息到Arduino
        messagesending_str = json.dumps(extracted_controls, ensure_ascii=False)
        ser.write(messagesending_str.encode('utf-8'))
        print(f"Sent: {messagesending_str}")

    except Exception as e:
        print(f"Failed to get a response from the LLM. Error: {e}")

# 持续监控文件更新
while True:
    try:
        # 获取当前文件大小
        current_file_size = os.path.getsize(file_path)
        
        # 检查文件大小是否增加，表示有新内容
        if current_file_size > last_file_size:
            # 打开文件并读取新内容
            with open(file_path, 'r') as file:
                # 将光标移到上次读取的位置
                file.seek(last_file_size)
                # 读取新内容
                new_content = file.read()
                
                # 将内容拆分为行（假设每个数据是一个新行）
                lines = new_content.splitlines()
                
                # 处理每一行
                for line in lines:
                    # 单独发送每一行
                    if line.strip():  # 检查行是否不为空
                        print(f"Sending new content to LLM: {line.strip()}")
                        send_message_to_llm(line.strip())
                
                # 更新上次读取的文件大小
                last_file_size = current_file_size
    
    except Exception as e:
        print(f"An error occurred: {e}")
        break

In [51]:
import langchain
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

# Initialize the OpenAI model
llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)

langchain.verbose = True

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Define LLM chains
first_prompt = ChatPromptTemplate.from_template("""
You will be given {flight_data}. 
The pilot is required to first execute a 45° steep turn to the right, completing a 360° turn, and then execute a 45° steep turn to the left, completing another 360° turn. 
Analyze flight data and determine the current phase of the steep turn with the data of angle of bank, pitch and heading. 
Return choose the current flight status as a string. Please strictly follow the criteria: 
1. Straight-and-Level Flying: Bank angle is close to 0 degrees, heading is stable.
2. Entry: Bank angle is increasing towards 45 degrees, heading is changing rapidly.
3. Steady-State Turn: Bank angle is maintained close to 45 degrees, heading is changing steadily, altitude is maintained.
4. Recovery: Heading is approaching 350 degrees, bank angle is decreasing towards 0 degrees, preparing to return to straight-and-level flight.
"""
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="flight_status")

def getMyPrompt():
    prompt_template = """Use the following pieces of context to answer the users question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    There are multiple answers, provide each answer and the source file for each answer.
    \n\n{context}\n\nQuestion: According to the flight_status{question}, retrieve the related operational and attention guidance. \n\n Answer in a paragraph:"""
    
    MyPrompt = PromptTemplate(
        template=prompt_template, input_variables=["context","question"]
    )
    return MyPrompt


chain_type_kwargs = {"prompt": getMyPrompt()}

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    input_key="flight_status",
    output_key="retrieved_guidance",
    chain_type_kwargs=chain_type_kwargs
)

third_prompt = ChatPromptTemplate.from_template(
    """
    Based on the {retrieved_guidance}, recommend specific joystick movements and the instruments to focus to optimize flight performance according to the current phase and maneuver.
    Output the required joystick movements as follows:
    flight_status= the current flight status
    pitch = forward/back/no
    roll = left/right/no
    instrument = Altimeter/Airspeed Indicator/Altimeter/Heading Indicator
    """
)

chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="answer")

overall_chain = SequentialChain(
    chains=[chain_one, qa, chain_three],
    input_variables=["flight_data"],
    output_variables=["flight_status", "retrieved_guidance", "answer"],
    verbose=True
)

# Example flight data input
flight_data = "{'timestamp': '2024-08-24T15:30:57.342Z', 'plane_altitude': '2400.9382626', 'ground_altitude': '217.0375361', 'plane_pitch_degrees': '-2.1871284', 'plane_bank_degrees': '-41.9298986', 'plane_heading_degrees_true': '257.0383409', 'plane_heading_degrees_magnetic': '266.2906136', 'airspeed_indicated': '116.6328278', 'airspeed_true': '120.8094788', 'ground_velocity': '119.0164008', 'vertical_speed': '200.3418303', 'incidence_beta': '0.1352000', 'acceleration_body_x': '21.5425801', 'acceleration_body_y': '16.1129594', 'acceleration_body_z': '0.0299524', 'incidence_alpha': '1.6090642'}"

# Run the entire chain with the provided flight data
res = overall_chain({"flight_data": flight_data})
print(res)
print(res["answer"])



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
You will be given {'timestamp': '2024-08-24T15:30:57.342Z', 'plane_altitude': '2400.9382626', 'ground_altitude': '217.0375361', 'plane_pitch_degrees': '-2.1871284', 'plane_bank_degrees': '-41.9298986', 'plane_heading_degrees_true': '257.0383409', 'plane_heading_degrees_magnetic': '266.2906136', 'airspeed_indicated': '116.6328278', 'airspeed_true': '120.8094788', 'ground_velocity': '119.0164008', 'vertical_speed': '200.3418303', 'incidence_beta': '0.1352000', 'acceleration_body_x': '21.5425801', 'acceleration_body_y': '16.1129594', 'acceleration_body_z': '0.0299524', 'incidence_alpha': '1.6090642'}. 
The pilot is required to first execute a 45° steep turn to the right, completing a 360° turn, and then execute a 45° steep turn to the left, completing another 360° turn. 
Analyze flight data and determine the current phase of the steep turn with the data of angle of bank, pitch and

In [28]:
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents import tool
from langchain.agents import AgentExecutor, create_react_agent, load_tools

# 解析YAML格式的数据
flight_data = """{'timestamp': '2024-08-24T15:31:42.625Z', 'plane_altitude': '2339.7345046', 'ground_altitude': '127.4123033', 'plane_pitch_degrees': '-2.3305856', 'plane_bank_degrees': '45.4910839', 'plane_heading_degrees_true': '25.2983926', 'plane_heading_degrees_magnetic': '34.2305463', 'airspeed_indicated': '122.4056396', 'airspeed_true': '126.5905533', 'ground_velocity': '127.5915478', 'vertical_speed': '16.6264361', 'incidence_beta': '0.2434822', 'acceleration_body_x': '-23.0330251', 'acceleration_body_y': '32.0754133', 'acceleration_body_z': '0.5025474', 'incidence_alpha': '2.9207101'}"""

# 初始化检索器
retriever = vector_store.as_retriever(k=3)

# Initialize memory
memory = ConversationBufferWindowMemory(k=500, memory_key="chat_history", return_messages=True)

# 从hub加载prompt
prompt = hub.pull("aviation")

@tool
def judge_flight_status(flight_data: str) -> str:
    """
    You will be given {flight_data}. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. Return the current flight status as a string.
    1. Straight-and-Level Flying: Bank angle is close to 0 degrees, heading is stable.
    2. Entry: Bank angle is increasing towards 40 degrees, heading is changing rapidly.
    3. Maintaining the Turn: Bank angle is maintained close to 45 degrees, heading is changing steadily, altitude is maintained.
    4. Recovery: Heading is approaching 340 degrees, bank angle is decreasing towards 0 degrees, preparing to return to straight-and-level flight.
    """
    flight_status = f"analyze_flight_data: {flight_data}"
    return flight_status

@tool
def retrieve_guidance(flight_status: str) -> str:
    """Based on the determined flight status, query the vector store to retrieve appropriate operational and attention guidance."""
    guidance_query = f"Retrieve guidance for {flight_status}."
    message = retriever.invoke(guidance_query)
    # Extract page content from each document in the message
    guidance = [doc.page_content for doc in message]
    # Convert the guidance list into a single string, joining with a suitable delimiter (e.g., newline, comma)
    guidance_str = "\n".join(guidance)  # Here, newline is used as a delimiter
    return guidance_str

@tool
def generate_answer(answer: str) -> str:
    """
    Based on the {retrieved_guidance}, recommend specific joystick movements and the instrument to focus to optimize flight performance according to the current phase and maneuver.
    Output the required joystick movements as follows:
    For pitch adjustments: "pitch = forward/back/no"
    For roll adjustments: "roll = left/right/no"
    Notice: "instrument = Altimeter/Airspeed Indicator/Altimeter/Heading Indicator"
    """
    return f"Here is the answer to the user question: {answer}"

# Register tools
tools = [judge_flight_status, retrieve_guidance, generate_answer]

# 初始化 LLM
llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)

# 创建 agent
agent = create_react_agent(llm, tools, prompt)

# 创建 agent 执行器
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True, handle_parsing_errors=True)

# 示例 flight data
# 你可以将 flight_data 直接传入
agent_executor.invoke({
    "input": flight_data
})



> Entering new AgentExecutor chain...
To answer the question, I need to determine the current phase of the steep turn using the provided flight data. I will first analyze the flight data to determine the current flight status.

Action: judge_flight_status
Action Input: {'plane_bank_degrees': '45.4910839', 'plane_heading_degrees_true': '25.2983926'}analyze_flight_data: {'plane_bank_degrees': '45.4910839', 'plane_heading_degrees_true': '25.2983926'}It seems there was a misunderstanding in the process. Let me correct that and proceed with the correct steps.

Action: judge_flight_status
Action Input: "{'plane_bank_degrees': '45.4910839', 'plane_heading_degrees_true': '25.2983926'}"analyze_flight_data: {'plane_bank_degrees': '45.4910839', 'plane_heading_degrees_true': '25.2983926'}It seems there was an error in the observation step. Let me correct that and proceed with the correct steps.

Action: judge_flight_status
Action Input: "plane_bank_degrees: 45.4910839, plane_heading_degrees_true

{'input': "{'timestamp': '2024-08-24T15:31:42.625Z', 'plane_altitude': '2339.7345046', 'ground_altitude': '127.4123033', 'plane_pitch_degrees': '-2.3305856', 'plane_bank_degrees': '45.4910839', 'plane_heading_degrees_true': '25.2983926', 'plane_heading_degrees_magnetic': '34.2305463', 'airspeed_indicated': '122.4056396', 'airspeed_true': '126.5905533', 'ground_velocity': '127.5915478', 'vertical_speed': '16.6264361', 'incidence_beta': '0.2434822', 'acceleration_body_x': '-23.0330251', 'acceleration_body_y': '32.0754133', 'acceleration_body_z': '0.5025474', 'incidence_alpha': '2.9207101'}",
 'chat_history': [],
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [59]:
#SimpleSequentialChain
# LangChain相关模块的导入
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

first_prompt = ChatPromptTemplate.from_template(
    """You will be given flight data. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. Return the current flight status as a string: {flight_data}"""
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, memory=memory, verbose=True)
 
retrieval_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(), verbose=True)

third_prompt = ChatPromptTemplate.from_template(
    """
    Based on the retrieved operational guidance, recommend specific joystick movements to optimize flight performance according to the current phase and maneuver.
    Output the required joystick movements as follows:
    For pitch adjustments: "pitch = forward/back/no"
    For roll adjustments: "roll = left/right/no"
    {answer}
    """
)

chain_three = LLMChain(llm=llm, prompt=third_prompt, memory=memory, verbose=True)

overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, retrieval_qa, chain_three],
    verbose=True
)

flight_data = """{'timestamp': '2024-08-24T15:30:56.260Z', 'plane_altitude': '2397.8084638', 'ground_altitude': '217.0375361', 'plane_pitch_degrees': '-1.8220291', 'plane_bank_degrees': '-42.0037138', 'plane_heading_degrees_true': '248.0215643', 'plane_heading_degrees_magnetic': '257.4566496', 'airspeed_indicated': '116.9294281', 'airspeed_true': '121.1400452', 'ground_velocity': '119.5141983', 'vertical_speed': '127.6741219', 'incidence_beta': '0.0871312', 'acceleration_body_x': '21.5418552', 'acceleration_body_y': '22.8557370', 'acceleration_body_z': '0.6362198', 'incidence_alpha': '2.5563520'}"""
res = overall_simple_chain.run(flight_data)
print(res)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: You will be given flight data. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. Return the current flight status as a string: {'timestamp': '2024-08-24T15:30:56.260Z', 'plane_altitude': '2397.8084638', 'ground_altitude': '217.0375361', 'plane_pitch_degrees': '-1.8220291', 'plane_bank_degrees': '-42.0037138', 'plane_heading_degrees_true': '248.0215643', 'plane_heading_degrees_magnetic': '257.4566496', 'airspeed_indicated': '116.9294281', 'airspeed_true': '121.1400452', 'ground_velocity': '119.5141983', 'vertical_speed': '127.6741219', 'incidence_beta': '0.0871312', 'acceleration_body_x': '21.5418552', 'acceleration_body_y': '22.8557370', 'acceleration_body_z': '0.6362198', 'incidence_alpha': '2.5563520'}

> Finished chain.
To determine the current phase of a steep turn, we need to analyze the angle of bank and

In [57]:
import langchain
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory

# Initialize the OpenAI model
llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)

langchain.verbose = True

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# First prompt: Determine flight status
first_prompt = ChatPromptTemplate.from_template(
    """You will be given {flight_data}. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. 
    Return the current flight status as a string."""
)

chain_one = LLMChain(
    llm=llm,
    prompt=first_prompt,
    output_key="flight_status",
)

# Second chain: Retrieve guidance based on flight status
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    chain_type="stuff",
    input_key="flight_status",
    output_key="retrieved_guidance"
)

# Third prompt: Recommend joystick movements based on retrieved guidance
third_prompt = ChatPromptTemplate.from_template(
    """
    Based on the {retrieved_guidance}, recommend specific joystick movements to optimize flight performance according to the current phase and maneuver.
    Output the required joystick movements as follows:
    For pitch adjustments: "pitch = forward/back/no"
    For roll adjustments: "roll = left/right/no"
    """
)

chain_three = LLMChain(
    llm=llm,
    prompt=third_prompt,
    output_key="answer",
)

# Set up the overall SequentialChain
overall_chain = SequentialChain(
    chains=[chain_one, qa, chain_three],
    input_variables=["flight_data"],
    output_variables=["flight_status", "retrieved_guidance", "answer"],
    verbose=True
)

# Example flight data input
flight_data = "{'timestamp': '2024-08-24T15:30:57.342Z', 'plane_altitude': '2400.9382626', 'ground_altitude': '217.0375361', 'plane_pitch_degrees': '-2.1871284', 'plane_bank_degrees': '-41.9298986', 'plane_heading_degrees_true': '257.0383409', 'plane_heading_degrees_magnetic': '266.2906136', 'airspeed_indicated': '116.6328278', 'airspeed_true': '120.8094788', 'ground_velocity': '119.0164008', 'vertical_speed': '200.3418303', 'incidence_beta': '0.1352000', 'acceleration_body_x': '21.5425801', 'acceleration_body_y': '16.1129594', 'acceleration_body_z': '0.0299524', 'incidence_alpha': '1.6090642'}"

# Run the entire chain with the provided flight data
res = overall_chain({"flight_data": flight_data})
print(res)
print(res["answer"])



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: You will be given {'timestamp': '2024-08-24T15:30:57.342Z', 'plane_altitude': '2400.9382626', 'ground_altitude': '217.0375361', 'plane_pitch_degrees': '-2.1871284', 'plane_bank_degrees': '-41.9298986', 'plane_heading_degrees_true': '257.0383409', 'plane_heading_degrees_magnetic': '266.2906136', 'airspeed_indicated': '116.6328278', 'airspeed_true': '120.8094788', 'ground_velocity': '119.0164008', 'vertical_speed': '200.3418303', 'incidence_beta': '0.1352000', 'acceleration_body_x': '21.5425801', 'acceleration_body_y': '16.1129594', 'acceleration_body_z': '0.0299524', 'incidence_alpha': '1.6090642'}. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. 
    Return the current flight status as a string.

> Finished chain.


> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new 

In [34]:
# LangChain相关模块的导入
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

pitch_schema = ResponseSchema(
    name="pitch",
    description="The pitch adjustment command for the aircraft in the current state, such as 'forward', 'back', or 'no'"
)

# Define the roll ResponseSchema
roll_schema = ResponseSchema(
    name="roll",
    description="The roll adjustment command for the aircraft in the current state, such as 'left', 'right', or 'no'"
)

# Combine the schemas into a list
response_schemas = [pitch_schema, roll_schema]

# Create the StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

first_prompt = ChatPromptTemplate.from_template(
    """You will be given flight data. Analyze flight data and determine the current phase of the steep turn with the data of angle of bank and heading. Return the current flight status as a string: {flight_data}"""
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, memory=memory, verbose=True)
 
retrieval_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(), verbose=True)

third_prompt = ChatPromptTemplate.from_template(
    """
    Based on the retrieved operational guidance, recommend specific joystick movements to optimize flight performance according to the current phase and maneuver.
    Output the required joystick movements in the following strict JSON format:
    {
      "pitch": "forward/back/no",
      "roll": "left/right/no"
    }
    """
)

chain_three = LLMChain(llm=llm, prompt=third_prompt, memory=memory, output_parser=output_parser, verbose=True)

overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, retrieval_qa, chain_three],
    verbose=True
)



flight_data = """{'timestamp': '2024-08-24T15:30:18.931Z', 'plane_altitude': '2282.3201289', 'ground_altitude': '117.8523424', 'plane_pitch_degrees': '-0.6233004', 'plane_bank_degrees': '36.5760190', 'plane_heading_degrees_true': '85.9190313', 'plane_heading_degrees_magnetic': '94.8891004', 'airspeed_indicated': '130.4407349', 'airspeed_true': '134.8597107', 'ground_velocity': '136.8825910', 'vertical_speed': '-148.4986210', 'incidence_beta': '-0.4252361', 'acceleration_body_x': '-19.3765291', 'acceleration_body_y': '18.8985063', 'acceleration_body_z': '-0.8903760', 'incidence_alpha': '1.0065350'}"""
res = overall_simple_chain.run(flight_data)
print(res)

ValidationError: 1 validation error for SimpleSequentialChain
__root__
  Chains used in SimplePipeline should all have one input, got memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history') verbose=True prompt=ChatPromptTemplate(input_variables=[], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n    Based on the retrieved operational guidance, recommend specific joystick movements to optimize flight performance according to the current phase and maneuver.\n    Output the required joystick movements in the following strict JSON format:\n    {{\n      "pitch": "forward/back/no",\n      "roll": "left/right/no"\n    }}\n    '))]) llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001AFC18503E0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001AFC17CB110>, model_name='gpt-4o-2024-08-06', temperature=0.0, openai_api_key='YOUR_KEY', openai_proxy='') output_pa